# CHATBOT TRAINER 

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
import random

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


ModuleNotFoundError: No module named 'tensorflow'

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documets
        documents.append((w,intent['tag']))
        # adding classes to our class
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# print(words,documents,classes,sep='\n')

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents),"documents")
print(len(classes),"classes")
print(len(words),"words")

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

34 documents
6 classes
58 words


In [7]:
# initializing training data

training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmetize each word - create base word, in attempt to represented related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is 0 for each tag and 1 for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

Training data created


<ipython-input-7-e6f544b669e5>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [8]:
# create model
# 3 layers. 1st: 128 neurons 2nd: 64 neurons 3rd: output layer equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

#compile model. Using Stochastic gradient descent with Nesterov accelerated gradient gives good result

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
7/7 [==============================] - 0s 3ms/step - loss: 1.8173 - accuracy: 0.1765
Epoch 2/200
7/7 [==============================] - 0s 0s/step - loss: 1.8349 - accuracy: 0.1471 
Epoch 3/200
7/7 [==============================] - 0s 0s/step - loss: 1.7362 - accuracy: 0.2647
Epoch 4/200
7/7 [==============================] - 0s 0s/step - loss: 1.6823 - accuracy: 0.2941 
Epoch 5/200
7/7 [==============================] - 0s 1ms/step - loss: 1.5866 - accuracy: 0.3235
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5315 - accuracy: 0.4118
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4745 - accuracy: 0.4412
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5152 - accuracy: 0.3529
Epoch 9/200
7/7 [==============================] - 0s 3ms/step - loss: 1.3548 - accuracy: 0.5294
Epoch 10/200
7/7 [==============================] - 0s 3ms/step - loss: 1.2920 - accuracy: 0.5000
Epoch 11/200
7/7 [============

7/7 [==============================] - 0s 1ms/step - loss: 0.0716 - accuracy: 0.9412
Epoch 168/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0147 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0289 - accuracy: 1.0000
Epoch 176/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0512 - accuracy: 0.9706
Epoch 177/200
7/7 [=====